##### 7.1 装饰器基本知识

In [7]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

target()

# target是 inner 的引用
target

running inner()


<function __main__.deco.<locals>.inner()>

##### 7.2 Python 何时执行装饰器
装饰器的一个关键特性是，它们在被装饰的函数定义后立即运行，通常是在 _导入时_

In [8]:
registry = []   # registry 保存被 @register 装饰的函数引用

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('f1')

@register
def f2():
    print('f2')

def f3():
    print('f3')

def main():
    print('main')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()


running register(<function f1 at 0x0000029E1A815550>)
running register(<function f2 at 0x0000029E1A815670>)
main
registry -> [<function f1 at 0x0000029E1A815550>, <function f2 at 0x0000029E1A815670>]
f1
f2
f3


##### 7.3 使用装饰器改进”策略“模式

In [9]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order): # 直接定义函数而不是类
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    return max(promo(order) for promo in promos)

##### 7.4 变量作用域

In [10]:
# b 是局部变量，因为在函数的定义体内为它赋值了
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

##### 7.5 闭包
闭包是延申了作用域的函数，其中包含函数定义体中引用、但是不在定义体中定义的非全局变量。

In [ ]:
# 计算移动平均值的类
class Averager():

    def __init__(self) -> None:
        self.series = []
    
    def __call__(self, new_value) -> float:
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [ ]:
avg = Averager()
print(avg(10))
print(avg(11))
print(avg(12))

In [ ]:
# 计算移动平均值的高阶函数
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

In [ ]:
avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(12))

注意到在 averager 函数中，series 是 _自由变量_ ，而 averager 的闭包延申到函数的作用于之外，包含自由变量 series 的绑定  
审查返回的 averager 对象，可以看到 Python 在 \_\_code__ 属性中保存了局部变量和自由变量的名称

In [ ]:
avg.__code__.co_varnames

In [ ]:
avg.__code__.co_freevars

series 的绑定在返回的 avg 函数的 \_\_closure__ 属性中，avg.\_\_closure__ 中的各个元素对应于 avg.\_\_code__.co_freevars 中的一个名称。这个元素是 cell 对象，有个 cell_contents 属性，保存着真正的指

In [ ]:
avg.__code__.co_freevars

In [ ]:
avg.__closure__

In [ ]:
avg.__closure__[0].cell_contents

##### 7.6 nonlocal 声明
前面实现 make_averager 函数的方法效率不高。我们可以只储存目前的总值和元素个数，然后使用这两个数计算均值。

In [ ]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1
        total += new_value
        return total / count

其中，我们在 averager中为 count 与 total 赋值了，这会把它们变成局部变量并在运行时抛出错误  
但在上一个实例中并未出现这个问题，这是因为我们没有给 series 赋值，我们只是调用 series.append，并把它传给 sum 和 len。  
为了解决这个问题，Python 3 引入了 nonlocal 声明。它的作用是把变量标记为自由变量，即时在函数中为变量赋予新值了，也会变成自由变量。如果为 nonlocal 声明的变量赋予新值，闭包中保存的绑定会更新

In [11]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

##### 7.7 实现一个简单的装饰器

In [12]:
# 一个简单的装饰器，输出函数的运行时间
import time

def clock(func):
    def clocked(*args):  # 他接受任意个定位参数
        t0 = time.perf_counter()    
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [13]:
import time

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n<2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =',factorial(6))


**************************************** Calling snooze(.123)
[0.13564990s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000040s] factorial(1) -> 1
[0.00002080s] factorial(2) -> 2
[0.00003780s] factorial(3) -> 6
[0.00005380s] factorial(4) -> 24
[0.00006930s] factorial(5) -> 120
[0.00008680s] factorial(6) -> 720
6! = 720


In [14]:
# 被装饰器装饰的 factorial 函数已经被替换成 clocked 的引用
factorial.__name__

'clocked'

In [ ]:
def f1():
    pass

f1

In [ ]:
factorial

In [15]:
# 改进后的 clock 装饰器
# 使用 funtools.warps 装饰器把相关属性从 func 复制到 clocked 中
import time
import functools

def clock(func):

    @functools.wraps(func)
    def clocked(*args, **keywords):  # 他接受任意个定位参数
        t0 = time.time()    
        result = func(*args, **keywords)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if keywords:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(keywords.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

##### 7.8 标准库中的装饰器
python 内置了三个用于装饰方法的函数：property、classmethod 和 staticmethod  
另一个常见的装饰器是 functools.warps，它的作用是协助构建行为良好的装饰器。  
标准库中最值得关注的两个装饰器是 lru_cache 和全新的 sigledispatch

###### 7.8.1 使用 functools.lru_cache 备忘录
functools.lru_cache 实现了备忘功能，这是一项优化技术，它把耗时的函数的结果保存起来，避免传入相同的参数时重复计算。LRU 为 ”Least Recently Used“ 的缩写，表明缓存不会无限制的增长，一段时间不用的缓存条目会被扔掉

In [16]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__ == '__main__':
    print(fibonacci(5))

[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00000000s] fibonacci(4) -> 3
[0.00000000s] fibonacci(5) -> 5
5


这样会非常浪费时间，但使用 lru_cache 可以显著改善性能

In [17]:
import functools

@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__ == '__main__':
    print(fibonacci(30))

[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00000000s] fibonacci(4) -> 3
[0.00000000s] fibonacci(5) -> 5
[0.00000000s] fibonacci(6) -> 8
[0.00000000s] fibonacci(7) -> 13
[0.00000000s] fibonacci(8) -> 21
[0.00000000s] fibonacci(9) -> 34
[0.00000000s] fibonacci(10) -> 55
[0.00000000s] fibonacci(11) -> 89
[0.00099802s] fibonacci(12) -> 144
[0.00000000s] fibonacci(13) -> 233
[0.00099802s] fibonacci(14) -> 377
[0.00000000s] fibonacci(15) -> 610
[0.00099802s] fibonacci(16) -> 987
[0.00000000s] fibonacci(17) -> 1597
[0.00099802s] fibonacci(18) -> 2584
[0.00000000s] fibonacci(19) -> 4181
[0.00099802s] fibonacci(20) -> 6765
[0.00000000s] fibonacci(21) -> 10946
[0.00099802s] fibonacci(22) -> 17711
[0.00000000s] fibonacci(23) -> 28657
[0.00099802s] fibonacci(24) -> 46368
[0.00000000s] fibonacci(25) -> 75025
[0.00099802s] fibonacci(26) -> 121393
[0.00000000s] fibonacci(27) -> 196418
[0.00099802s] fibonacci(28) -

lru_cache 可以使用两个可选的参数来配置。它的签名是： 
funtools.lru_cache(maxsize=128, typed=False)
* maxsize   指定存储多少个调用的结果，缓存满了之后旧的结果会被扔掉，腾出空间。为了最佳性能 maxsize 应设为 2 的幂。
* typed     如果为 True，把不同参数类型得到的结果分开保存，即将 1 与 1.0 区分开

还有一个需要注意的是，lru_cache 使用字典储存结果，并且键根据传入的定位参数和关键字参数创建，所以被 lru_cache 装饰的函数，它的所有参数都必须是可散列的。

###### 7.8.2 单分派泛函数 funtools.singledispatch

In [18]:
# 生成不同类型 Python 对象的 HTML
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

这个函数适用于任何 Python 类型，但我们想做个扩展，让他使用特别的方式显示这些类型。
* str: 把内部的换行符替换为 '\<br>\n'; 不使用 \<pre>，而是使用 \<p>
* int: 以十进制和十六进制显示数字
* list: 输出一个 HTML 列表，根据各个元素的类型进行格式化

In [21]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch # @singledispatch 标记处理 object 类型的基函数
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral) # numbers.Integral 是 int 的虚拟超类
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence) # 可以叠放多个 register 装饰器，让同一个函数支持不同类型
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

# 只要可能，尽量使用抽象基类注册而不处理具体实现，这样代码的兼容性更广泛

##### 7.9 叠放装饰器

##### 7.10 参数化装饰器
可以创建装饰器工厂函数，把参数传给它，返回一个装饰器，然后再把它应用到要装饰的函数上

In [24]:
registry = []   

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('f1')


print('main')
print('registry ->', registry)
f1()
    

running register(<function f1 at 0x0000029E1A8155E0>)
main
registry -> [<function f1 at 0x0000029E1A8155E0>]
f1


###### 7.10.1 一个参数化的注册装饰器
为了让 regiser 同时具备可选的注册和注销功能，需要设置一个参数，将其设为 false 时，不注册装饰的函数。实现方式如下

In [25]:
registry = set()    # registry 现在是一个 set 对象，这样添加和删除函数的速度更快

def register(active=True):  # registry 接受一个可选的关键字参数
    def decorate(func): # decorate 这个内部函数是真正的装饰器：它的参数是一个函数。
        print('running register(active=%s)->decorate(%s)' % (active, func))
        if active:  # 只有 active 的值是 True 时才注册 func，注意此处 active 从闭包获取
            registry.add(func)
        else:
            registry.discard(func)

        return func # decorate 是装饰器，所以返回函数
    return decorate # register 是工厂函数，返回 decorate

@register(active=False)
def f1():
    print('f1')

@register() # register 是工厂函数必须作为函数调用，即需要 ()
def f2():
    print('f2')

def f3():
    print('f3')

running register(active=False)->decorate(<function f1 at 0x0000029E1A830EE0>)
running register(active=True)->decorate(<function f2 at 0x0000029E1A815670>)


###### 参数化 clock 装饰器

In [29]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'
def clock(fmt = DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):  # 他接受任意个定位参数
            t0 = time.time()    
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result) # result 是 _result 的字符串表示，用于显示
            print(fmt.format(**locals()))   # 这里使用 **locals() 是为了在 fmt 中引用 clocked 的局部变量。
            # print(locals())
            return _result
        return clocked
    return decorate

if __name__ == '__main__':

    @clock()
    def sonnze(seconds):
        time.sleep(seconds)
    
    for i in range(3):
        sonnze(.123)

[0.13188124s] sonnze(0.123) -> None
[0.12366486s] sonnze(0.123) -> None
[0.13665342s] sonnze(0.123) -> None


In [30]:
@clock('{name}: {elapsed}s')
def sonnze(seconds):
    time.sleep(seconds)

for i in range(3):
    sonnze(.123)

sonnze: 0.13677716255187988s
sonnze: 0.13685107231140137s
sonnze: 0.13715124130249023s
